## 爬取目标

<div align=center>
    <img src="../../../images/fb0cc800f904f30931b46f0af6dd6c257e0a709d67c0dec84766a0f3562b9be4.png" width=75%>
    </img>
</div>

- 1. 提取每一页的电影列表,以及每个电影的详情页

- 2. 提取每部电影的`名称`、`封面`、`类别`、`上映时间`、`评分`、`剧情简介`

- 3. 存入数据库

- 4. 多进程

### 1. 提取每一页的电影列表,以及每个电影的详情页

In [3]:
import requests
import re
from pyquery import PyQuery as pq

url = 'https://ssr1.scrape.center'

res = requests.get(url)
res

<Response [200]>

In [44]:
with open('11基础案例HTML.html', 'w') as fp:
    fp.write(res.text)

#### 使用 css 选择器

In [50]:
import re

movie_name = re.search('<h2.*?>(.*"?)</h2>', res.text)
movie_name

<re.Match object; span=(1933, 2004), match='<h2 data-v-7f856186="" class="m-b-sm">霸王别姬 - Fare>

In [ ]:
movies_page = re.search('<ul.*?li.*?href="(.*?)".*?</li>', res.text)
print(movies_page)

In [20]:
movie_name_list = re.findall('<h2.*?>(.*"?)</h2>', res.text)
len(movie_name_list),movie_name_list

(10,
 ['霸王别姬 - Farewell My Concubine',
  '这个杀手不太冷 - Léon',
  '肖申克的救赎 - The Shawshank Redemption',
  '泰坦尼克号 - Titanic',
  '罗马假日 - Roman Holiday',
  '唐伯虎点秋香 - Flirting Scholar',
  '乱世佳人 - Gone with the Wind',
  '喜剧之王 - The King of Comedy',
  '楚门的世界 - The Truman Show',
  '狮子王 - The Lion King'])